<a href="https://colab.research.google.com/github/Girijanath-parida/RAG-Documents/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install langchain chromadb PyPDF2 langchain-experimental langchain_community openai Pypdf chromadb langchain-groq langchain-openai sentence_transformers langchain-chroma

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.3/204.3 kB 1.4 MB/s eta 0:00:00


In [11]:
import openai
import os


os.environ['OPENAI_API_KEY'] = os.getenv("OPENAI_API_KEY")
os.environ['GROQ_API_KEY'] = os.getenv("GROQ_API_KEY")
groq_api_key=os.getenv("GROQ_API_KEY")

from langchain_groq import ChatGroq

# Document loader
from langchain.document_loaders import PyPDFLoader

# Vector Store
from langchain_community.vectorstores import Chroma

#llm
from langchain_openai import OpenAI



In [13]:

loader = PyPDFLoader("/content/Python Machine Learning.pdf")
pages = loader.load()

In [14]:
len(pages)

771

In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size =500,chunk_overlap  = 100)
docs=text_splitter.split_documents(pages)

In [16]:
len(docs)

3524

In [17]:
docs[63].page_content

'Training the GAN model 638\nImproving the quality of synthesized images using  \na convolutional and Wasserstein GAN 646\nTransposed convolution 647\nBatch normalization 648\nImplementing the generator and discriminator 651\nDissimilarity measures between two distributions 657\nUsing EM distance in practice for GANs 661\nGradient penalty 662\nImplementing WGAN-GP to train the DCGAN model 663\nMode collapse 667\nOther GAN applications 669\nSummary 670\nChapter 18: Reinforcement Learning for Decision'

In [18]:
import chromadb

In [19]:
client=chromadb.PersistentClient(path="./docs_cache")

In [ ]:
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings
embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
db = Chroma.from_documents(docs, embedding_function)

In [ ]:
query = input()
response = db.similarity_search(query)
response

Build RAG using above


In [13]:

#from langchain.chat_models import ChatOpenAI
#llm=ChatOpenAI(temprature=0.0, model='gpt-3.5-turbo')
llm=ChatGroq(groq_api_key=groq_api_key,model_name="Llama3-8b-8192")

In [15]:
retriever=db.as_retriever()

In [33]:
from langchain.prompts import PromptTemplate
prompt_template = """Please answer questions related to python language. If you don't know the answer, just say that "I am unable to answer that",
CONTEXT: {context}
Question: {question}"""
PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [34]:
from langchain.chains import RetrievalQA
chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    input_key = 'query',
    return_source_documents=True,
    chain_type_kwargs = {"prompt":PROMPT},
    verbose=True)

In [ ]:
query = input()
response = chain(query)
response

In [ ]:
query = input()
response = chain(query)
response

In [ ]:
query = input()
response = chain(query)
response
